#### Importing libraries

In [14]:
import xlwings as xw
from datetime import datetime
import openpyxl
from openpyxl.utils import get_column_letter
import numpy as np
import re
import pandas as pd


#### Opening files

In [32]:
#Reading files
user = input("Enter your laptop user: ")
# Get the current date
current_date = datetime.now().strftime("%Y%m%d")

gl = xw.Book(r'C:\Users\{user}\Downloads\general_ledger-{çurrent_day}.xlsx'.format(user=user, çurrent_day=current_date))
sec_dep_det = xw.Book(r'C:\Users\{user}\Downloads\security_deposit_funds_detail-{çurrent_day}.xlsx'.format(user=user,çurrent_day=current_date))

# Indicating worksheet
gl_worksheet = gl.sheets['Sheet1']
sec_dep_det_worksheet = sec_dep_det.sheets['Sheet1']

#### Diagnosis 1

##### Including adjusted payee names

In [33]:
# Defining rows to calculate values
first_row_of_values = gl.sheets['Sheet1'].range('a'+ str(gl.sheets['Sheet1'].cells.last_cell.row)).end('up').end('up').end('up').row + 2 # first row of values
last_row_of_values = gl.sheets['Sheet1'].range('a'+ str(gl.sheets['Sheet1'].cells.last_cell.row)).end('up').end('up').row - 1  # last row of values



# Creating columns to adjust payee names
gl.sheets['Sheet1'].api.Columns(4).Insert() # Inserting column
gl.sheets['Sheet1'].api.Columns(4).Insert() # Inserting another column to include numerical index for payee names


# Including adjusted payee names

# Read values from column C and remove spaces using pandas
data_range_for_payee = gl.sheets['Sheet1'].range('c{}:c{}'.format(first_row_of_values, last_row_of_values))
column_c_values = data_range_for_payee.value

# Convert to a pandas DataFrame for efficient processing
df = pd.DataFrame(column_c_values, columns=['Column_C'])

# Remove spaces and handle empty values
df['Column_D'] = df['Column_C'].apply(lambda x: x.replace(" ", "") if isinstance(x, str) else "No Payee Name")

# Write the modified data back to column D & E (including numerical index)
gl.sheets['Sheet1'].range('D{}'.format(first_row_of_values)).value = dict(df['Column_D'])
gl.sheets['Sheet1'].range('E{}'.format(first_row_of_values - 2)).value = 'Payee'





##### Adding net column (debt - credit)

In [34]:
# Adding net column (debt - credit)
gl.sheets['Sheet1'].api.Columns(10).Insert() # Inserting column

# Set the formula for the entire range of cells in column J
range_formula = '=H{}:H{}-I{}:I{}'.format(first_row_of_values, last_row_of_values, first_row_of_values, last_row_of_values)
# Applying the formula to the entire column from the starting row to the ending row
gl.sheets['Sheet1'].range('J{}:J{}'.format(first_row_of_values, last_row_of_values)).formula = range_formula 
gl.sheets['Sheet1'].range('J{}'.format(first_row_of_values - 2)).value = 'Net'


##### Funds detail tenant name

In [35]:
sec_dep_det.sheets['Sheet1'].api.Columns(3).Insert() # Inserting column
sec_dep_det.sheets['Sheet1'].api.Columns(4).Insert() # Inserting another column to include numerical index for tenant names



# Defining rows to calculate values
first_row_of_values_sec_dep = sec_dep_det.sheets['Sheet1'].range('a'+ str(sec_dep_det.sheets['Sheet1'].cells.last_cell.row)).end('up').end('up').end('up').row + 1 # first row of values
last_row_of_values_sec_dep = sec_dep_det.sheets['Sheet1'].range('a'+ str(sec_dep_det.sheets['Sheet1'].cells.last_cell.row)).end('up').end('up').row  # last row of values


# Read the data into a pandas DataFrame
column_data = sec_dep_det.sheets['Sheet1'].range('b{}:b{}'.format(first_row_of_values_sec_dep, last_row_of_values_sec_dep)).value
df = pd.DataFrame(column_data, columns=['Names'])

# Rearrange the names efficiently using pandas
df['Names'] = df['Names'].str.split(', ').str[::-1].str.join(' ')
df['Names'] = df['Names'].str.replace(r'\s+', '', regex=True)

# Write the updated data back to Excel
sec_dep_det.sheets['Sheet1'].range('c{}:c{}'.format(first_row_of_values_sec_dep, last_row_of_values_sec_dep)).value = dict(df['Names'])


##### Creating pivot table

In [36]:
# Open a new Excel workbook
diagnostics_1_wb = xw.Book()

# Property Name
diagnostics_1_wb.sheets['Sheet1'].range('a1:g1').api.Font.Bold = True
gl.sheets['Sheet1'].range('m{}:m{}'.format(first_row_of_values, last_row_of_values)).copy()
diagnostics_1_wb.sheets['Sheet1'].range('A2').paste()
diagnostics_1_wb.sheets['Sheet1'].range('A1').value = 'Property Name'

# Payee

gl.sheets['Sheet1'].range('e{}:e{}'.format(first_row_of_values, last_row_of_values)).copy()
diagnostics_1_wb.sheets['Sheet1'].range('b2').paste()
diagnostics_1_wb.sheets['Sheet1'].range('b1').value = 'Payee'

# SUM of net
gl.sheets['Sheet1'].range('o{}:o{}'.format(first_row_of_values, last_row_of_values)).formula = "=SUMIFS($J${first_row}:$J${last_row},$E${first_row}:$E${last_row},E{first_row},$M${first_row}:$M${last_row},M{first_row})".format(first_row=first_row_of_values, last_row= last_row_of_values)
values_list = gl.sheets['Sheet1'].range('o{}:o{}'.format(first_row_of_values, last_row_of_values)).value
result_dict = {index: value for index, value in enumerate(values_list)}
diagnostics_1_wb.sheets['Sheet1'].range('c2').value = result_dict
diagnostics_1_wb.sheets['Sheet1'].range('c:c').api.Delete() # Deleting column C
diagnostics_1_wb.sheets['Sheet1'].range('c1').value = 'Sum of Net'

# SD FUNDS DETAIL
# diagnostics_1_wb last row
last_row = diagnostics_1_wb.sheets[0].range('A' + str(diagnostics_1_wb.sheets[0].cells.last_cell.row)).end('up').row

# formula base string
formula_base = (
    '=SUMIFS(\'[security_deposit_funds_detail-{current_date}.xlsx]Sheet1\'!$H${first_row_of_values_sec_dep}:$H${last_row_of_values_sec_dep},'
    '\'[security_deposit_funds_detail-{current_date}.xlsx]Sheet1\'!$I${first_row_of_values_sec_dep}:$I${last_row_of_values_sec_dep},'
    '{{}},'
    '\'[security_deposit_funds_detail-{current_date}.xlsx]Sheet1\'!$D${first_row_of_values_sec_dep}:$D${last_row_of_values_sec_dep},{{}})'
).format(current_date=current_date, first_row_of_values_sec_dep=first_row_of_values_sec_dep, last_row_of_values_sec_dep=last_row_of_values_sec_dep)

# Generate a list of formulas for the entire range
formulas = [formula_base.format(f'A{i}', f'B{i}') for i in range(1, last_row + 1)]

# Set the formulas for the entire column A at once
column_range = diagnostics_1_wb.sheets[0].range(f'd1:d{last_row}')
column_range.options(transpose=True).formula = formulas
column_range.options(transpose=True).value = column_range.value
diagnostics_1_wb.sheets['Sheet1'].range('d1').value = 'SD Funds Detail'


# Difference
# Get the data from Excel into a DataFrame
sheet = diagnostics_1_wb.sheets[0]  
data_range = sheet.range('c1').expand('table')  # The data starts from C1 and is structured as a table
df = data_range.options(pd.DataFrame, index=False, header=True).value  # Read data into DataFrame

# Calculate the difference between columns 'Sum of Net' and 'SD Funds Detail'
df['Difference'] = round(df['Sum of Net'] + df['SD Funds Detail'])

# Write the results back to Excel
start_cell = data_range.last_cell.offset(row_offset=2, column_offset=0)  
sheet.range(diagnostics_1_wb.sheets[0]['E1']).value = df['Difference']  # Write DataFrame back to Excel, starting in E1
# Delete the entire column
diagnostics_1_wb.sheets[0].api.Columns('e').Delete()



# Save the workbook as an Excel file
diagnostics_1_wb.save(r'C:\Users\{user}\Downloads\diagnosis_1.xlsx'.format(user=user))

# Close the workbook
diagnostics_1_wb.close()
gl.close()
sec_dep_det.close()




#### Diagnosis 2


In [20]:
gl = xw.Book(r'C:\Users\{user}\Downloads\general_ledger-{çurrent_day} (1).xlsx'.format(user=user, çurrent_day=current_date))
gl_worksheet = gl.sheets['Sheet1']


# Defining rows to calculate values
first_row_of_values = gl.sheets['Sheet1'].range('a'+ str(gl.sheets['Sheet1'].cells.last_cell.row)).end('up').end('up').end('up').row + 2 # first row of values
last_row_of_values = gl.sheets['Sheet1'].range('a'+ str(gl.sheets['Sheet1'].cells.last_cell.row)).end('up').end('up').row - 1  # last row of values



# Creating columns to adjust payee names
gl.sheets['Sheet1'].api.Columns(4).Insert() # Inserting column
gl.sheets['Sheet1'].api.Columns(4).Insert() # Inserting another column to include numerical index for payee names


# Including adjusted payee names

# Read values from column C and remove spaces using pandas
data_range_for_payee = gl.sheets['Sheet1'].range('c{}:c{}'.format(first_row_of_values, last_row_of_values))
column_c_values = data_range_for_payee.value

# Convert to a pandas DataFrame for efficient processing
df = pd.DataFrame(column_c_values, columns=['Column_C'])

# Remove spaces and handle empty values
df['Column_D'] = df['Column_C'].apply(lambda x: x.replace(" ", "") if isinstance(x, str) else "No Payee Name")

# Write the modified data back to column D & E (including numerical index)
gl.sheets['Sheet1'].range('D{}'.format(first_row_of_values)).value = dict(df['Column_D'])
gl.sheets['Sheet1'].range('E{}'.format(first_row_of_values - 2)).value = 'Payee'

# Adding net column (debt - credit)
gl.sheets['Sheet1'].api.Columns(10).Insert() # Inserting column

# Set the formula for the entire range of cells in column J
range_formula = '=H{}:H{}-I{}:I{}'.format(first_row_of_values, last_row_of_values, first_row_of_values, last_row_of_values)
# Applying the formula to the entire column from the starting row to the ending row
gl.sheets['Sheet1'].range('J{}:J{}'.format(first_row_of_values, last_row_of_values)).formula = range_formula 
gl.sheets['Sheet1'].range('J{}'.format(first_row_of_values - 2)).value = 'Net'


# Open a new Excel workbook
diagnostics_1_wb = xw.Book()

# Property Name
diagnostics_1_wb.sheets['Sheet1'].range('a1:g1').api.Font.Bold = True
gl.sheets['Sheet1'].range('m{}:m{}'.format(first_row_of_values, last_row_of_values)).copy()
diagnostics_1_wb.sheets['Sheet1'].range('A2').paste()
diagnostics_1_wb.sheets['Sheet1'].range('A1').value = 'Property Name'

# Payee

gl.sheets['Sheet1'].range('e{}:e{}'.format(first_row_of_values, last_row_of_values)).copy()
diagnostics_1_wb.sheets['Sheet1'].range('b2').paste()
diagnostics_1_wb.sheets['Sheet1'].range('b1').value = 'Payee'

# SUM of net
gl.sheets['Sheet1'].range('o{}:o{}'.format(first_row_of_values, last_row_of_values)).formula = "=SUMIFS($J${first_row}:$J${last_row},$E${first_row}:$E${last_row},E{first_row},$M${first_row}:$M${last_row},M{first_row})".format(first_row=first_row_of_values, last_row= last_row_of_values)
values_list = gl.sheets['Sheet1'].range('o{}:o{}'.format(first_row_of_values, last_row_of_values)).value
result_dict = {index: value for index, value in enumerate(values_list)}
diagnostics_1_wb.sheets['Sheet1'].range('c2').value = result_dict
diagnostics_1_wb.sheets['Sheet1'].range('c:c').api.Delete() # Deleting column C
diagnostics_1_wb.sheets['Sheet1'].range('c1').value = 'Sum of Net'


# Save the workbook as an Excel file
diagnostics_1_wb.save(r'C:\Users\{user}\Downloads\diagnosis_2.xlsx'.format(user=user))

# Close the workbook
gl.close()
diagnostics_1_wb.close()


#### Diagnosis 3

#### Notes

- No cambiarle el nombre a los archivos que se descargan de appfolio

- no tener en la carpeta de descargas archivos previos a este proceso con los siguientes nombres
general_ledger-20231110

    - security_deposit_funds_detail-20231110

    - delinquency-20231110